In [1]:
from network import Network
from conv import Conv
from dense import Dense
from activation import Relu
from metrics import log_loss, log_loss_grad, softmax
import math
import numpy as np
import random

In [2]:
random.seed(0)
FILE = "fashion-mnist_train.csv"
with open(f"../data/{FILE}") as f:
    examples = f.read().strip().split("\n")[1:]

train, valid, test = [], [], []
valid_cutoff = .8
test_cutoff = .9

for example in examples:
    label, pixels = example.split(",", 1)
    label = int(label)
    if label > 4:
        continue
    pixels = [int(p) for p in pixels.split(",")]
    data = np.asarray(pixels, dtype="int32")
    data = data.reshape((1,28,28))
    # Scale values from -1 to 1
    data = ((data / 255) - .5) / .5
    target = np.zeros((1, 5))
    target[0,label] = 1
    row = (data, target, )

    split = random.random()
    if split > test_cutoff:
        test.append(row)
    elif split > valid_cutoff:
        valid.append(row)
    else:
        train.append(row)

In [3]:
class TestNet(Network):
    def __init__(self):
        self.conv1 = Conv(input_channels=1, output_channels=3, kernel_x=3, kernel_y=3)
        self.conv2 = Conv(input_channels=3, output_channels=1, kernel_x=3, kernel_y=3)
        self.dense = Dense(input_size=24 ** 2, output_size=5, activation=False)
        self.last_input = None

        super().__init__()

    def forward(self, x):
        x = self.conv1.forward(x)
        x = self.conv2.forward(x)
        x = x.reshape(1, math.prod(x.shape))
        x = self.dense.forward(x)
        return x

    def backward(self, grad, lr):
        grad = self.dense.backward(grad, lr)
        grad = self.conv2.backward(grad, lr)
        grad = self.conv1.backward(grad, lr)

In [4]:
net = TestNet()
lr = 5e-4
epochs = 20

for epoch in range(epochs):
    epoch_loss = 0
    for i, img in enumerate(train):
        image, target = img
        batch = net.forward(image)

        grad = log_loss_grad(softmax(batch), target)
        epoch_loss += log_loss(softmax(batch), target)
        net.backward(grad, lr)

    print(f"Epoch {epoch} loss: {epoch_loss / len(train)}")
    match = np.zeros(len(valid))
    for i, img in enumerate(valid):
        image, target = img
        valid_pred = net.forward(image)
        match[i] = np.argmax(valid_pred) == np.argmax(target)

    print(f"Epoch {epoch} valid accuracy: {match.sum() / match.shape[0]}")

Epoch 0 loss: 0.9882971977021661
Epoch 0 valid accuracy: 0.7571810287241149
Epoch 1 loss: 0.671348362336631
Epoch 1 valid accuracy: 0.7852371409485638
Epoch 2 loss: 0.6007674499802047
Epoch 2 valid accuracy: 0.7969271877087508
Epoch 3 loss: 0.5640594744083588


KeyboardInterrupt: 